## Moteur d'émulation de Miasm

Ce petit document est une introduction aux API du moteur d'émulation de Miasm.
Ce moteur d'émulation est la plupart du temps appelé **jitter** dans l'ensemble du code du projet (car il se base sur des méthodes de [JiT](https://en.wikipedia.org/wiki/Just-in-time_compilation)).

La première chose à faire est d'obtenir une instance du *jitter*. Pour ce faire, nous allons passer par un object intermédiaire, `Machine`. Il va nous permettre d'instancier un ensemble d'élément lié à une architecture donnée supportée par Miasm (x86, arm, mips, ...).

In [1]:
from miasm2.analysis.machine import Machine
Machine.available_machine()

['arml',
 'armb',
 'armtl',
 'armtb',
 'sh4',
 'x86_16',
 'x86_32',
 'x86_64',
 'msp430',
 'mips32b',
 'mips32l',
 'aarch64l',
 'aarch64b',
 'ppc32b']

In [3]:
machine = Machine("x86_32")
print machine.name

x86_32


`Machine` propose plusieurs interfaces:
* `jitter`: un moteur d'émulation
* `dis_engine`: un moteur de désassemblage
* `ir`: un moteur pour l'utilisation du langage intermédiaire (hors scope)
* `ira`: quasiment comme `ir` (hors scope)
* `mn`: l'objet contenant les APIs de bas niveau pour une architecture donnée (désassemblage d'une ligne, assemblage, etc.)
* `gdbserver`: un serveur GDB (parlant le protocol GDB, à lié avec une instance de jitter)

Pour obtenir un jitter, nous allons devoir préciser le moteur de jit sur lequel nous souhaitons nous reposer.
Ici, cela n'a pas grande importance, mais les options possibles sont :
* "python": un moteur basé sur de l'exécution entièrement en Python, lent
* "gcc": un moteur basé sur du JiT via le compilateur GCC, rapide
* "llvm": un moteur basé sur du JiT via le compilateur LLVM, rapide

In [4]:
jitter = machine.jitter("python")
print jitter

Notre *jitter* instancié, il représente maintenant une "coquille vide", contenant des registres (un *cpu* avec au départ des registres qui valent tous 0) et une mémoire virtuelle (*vm*, pour *Virtual Memory*).
Pour manipuler les registres :

In [5]:
jitter.cpu.EAX

0L

In [6]:
jitter.cpu.EAX = 1
jitter.cpu.EAX

1L

In [7]:
# GPReg : General Purpose registers
regs = jitter.cpu.get_gpreg()
print regs

{'FS': 0L, 'RCX': 0L, 'af': 0L, 'MM5': 0L, 'MM4': 0L, 'zf': 0L, 'MM6': 0L, 'MM1': 0L, 'cf': 0L, 'MM3': 0L, 'MM2': 0L, 'XMM10': 0L, 'XMM11': 0L, 'XMM12': 0L, 'XMM13': 0L, 'XMM14': 0L, 'nf': 0L, 'DS': 0L, 'R14': 0L, 'R15': 0L, 'R12': 0L, 'R13': 0L, 'R10': 0L, 'R11': 0L, 'RIP': 0L, 'pf': 0L, 'tsc2': 286331153L, 'MM7': 0L, 'RAX': 1L, 'tsc1': 572662306L, 'GS': 0L, 'RBX': 0L, 'df': 0L, 'R8': 0L, 'R9': 0L, 'RDX': 0L, 'MM0': 0L, 'CS': 0L, 'RBP': 0L, 'RDI': 0L, 'ES': 0L, 'RSI': 0L, 'SS': 0L, 'of': 0L, 'XMM2': 0L, 'XMM3': 0L, 'XMM0': 0L, 'XMM1': 0L, 'XMM6': 0L, 'XMM7': 0L, 'XMM4': 0L, 'XMM5': 0L, 'XMM8': 0L, 'XMM9': 0L, 'RSP': 0L, 'XMM15': 0L}


La mémoire est manipulée grâce à `jitter.vm`. C'est au départ une coquille vide, sans aucun contenu

In [8]:
jitter.vm

Addr               Size               Access Comment

Il est alors possible d'y ajouter des pages mémoires, à des adresses arbitraires, de taille arbitraire (ie. pas d'arrondi à 0x1000 comme ce qui peut se produire sur certaines architectures réelles), avec du contenu, des droits d'accès ainsi qu'un commentaire (optionnel).

In [9]:
from miasm2.jitter.csts import PAGE_READ, PAGE_WRITE, PAGE_EXEC
jitter.vm.add_memory_page(0x1000, PAGE_READ | PAGE_WRITE | PAGE_EXEC, "\x00" * 0x1000, "test page")
jitter.vm.add_memory_page(0x112233, PAGE_READ | PAGE_WRITE | PAGE_EXEC, "\x00" * 0x666, "pas d'arrondi")
jitter.vm

Addr               Size               Access Comment
0x1000             0x1000             RWX    test page
0x112233           0x666              RWX    pas d'arrondi

La mémoire peut être accèder, en écriture et en lecture, depuis le Python :

In [10]:
jitter.vm.get_mem(0x1000, 0x10)

'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [11]:
jitter.vm.set_mem(0x1000, "toto")
jitter.vm.get_mem(0x1000, 0x10)

'toto\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

Quelques fonctions sont présentes pour faciliter les tâches les plus courantes, comme récupérer une chaîne de caractère ANSI:

In [12]:
jitter.get_str_ansi(0x1000)

'toto'

Nous allons maintenant placer du code dans l'émulateur, puis l'éxecuter en logguant l'état des registres (`log_regs`) et l'instruction correspondante (`log_mn`).

In [13]:
jitter.vm.set_mem(0x1000, "B844332211C3".decode('hex'))
jitter.jit.log_mn = True
jitter.jit.log_regs = True

L'émulation est initié à l'adresse 0x1000 via `jitter.init_run(adresse)` et est lancé par `jitter.continue_run()`.
Dans certain cas d'arrêt de l'émulation, c'est cette même API qui nous permettra de continuer l'exécution en reprenant où elle en était.

In [ ]:
jitter.init_run(0x1000)
jitter.continue_run()
# /!\ l'état des registres n'est pas affiché dans Jupyter, c'est normal !
# La sortie n'est pas rattrapée, elle est donc présente dans votre terminal de lancement de jupyter

# /!\ Il est normal de finir sur un RuntimeError

L'exécution "MOV, RET" fini par lever une exception. C'est un comportement tout à fait normal, nous n'avons pas placé de stack en mémoire, et nous essayons de faire un RET. Un déréférencement de 0 intervient donc, ce qui se traduit par le message :
`"WARNING: address 0x0 is not mapped in virtual memory"`

### Exercice 1
En redémarrant avec une nouvelle instance de jitter :
1. Ajoutez une page mémoire qui servira de stack
1. Faites pointer le pointeur de stack dessus (attention, la stack "marche à l'envers")
1. Ajoutez une valeur dans la stack, pour qu'elle soit utilisée par le RET
1. Lancez l'émulation, et vérifiez que vous finissez bien sur l'adresse de votre choix (il n'y a pas de code à cet endroit)

### Exercice 1 - Spoiler
Attention, spoiler ! Réalisez d'abord l'exercice 1 avec les APIs présentées ci-dessus.
Une fois que tout marche, sélectionnez le texte ci-dessous pour voir de nouvelles APIs (blanc sur blanc)

<pre style="color:white">
# Init a stack and set stack pointer accordingly (architecture agnostic way of writing)
jitter.init_stack()

# Push 0x1000 on the stack
jitter.push_uint32_t(0x1000)

# Pop 0x1000 from the stack
print jitter.pop_uint32_t()
</pre>

Il est aussi possible de poser des breakpoints. Ces breakpoints sont associés à une adresse, et vont appeler une callback Python lorsque cette adresse est atteinte par l'émulateur. Elle peuvent retourner :
- True: l'exécution continuera
- Tout sauf True: l'exécution s'arrête et la main est redonnée en sortie de `continue_run`. La valeur renvoyée par `continue_run` est celle renvoyé par le breakpoint

La syntaxe est la suivante :

In [15]:
def hello_world(jitter):
    print "Hello, world!"
    print "EAX value is %d", jitter.cpu.EAX
    # Stop execution right here
    return False

jitter.add_breakpoint(0x1005, hello_world)

### Exercice 2
Refaite l'exercice 1, mais cette fois-ci, ajouter un breakpoint sur l'adresse de retour que vous avez créé et mis sur la stack.
Ce breakpoint permettra de vérifier que nous avons bien atteint le RET.

Dans Miasm, un mécanisme similaire est très souvent utilisé. Il est nommé "code_sentinelle", et permet de récupérer la main à la fin de l'exécution d'une fonction, d'un shellcode, etc.

Le mécanisme des breakpoints permet cependant de faire bien plus; par exemple, il peut aussi être utilisé pour simuler les effets de bords d'un bout de code, comme une fonction. Il suffit pour cela d'avoir un breakpoint à l'adresse de la fonction, d'effectuer dedans les effets de bords voulus, puis de laisser l'exécution continuer plus loin. Ce dernier effet de bord est obtenu en modifiant la valeur de `jitter.pc` (pour *Program Counter*).

Par exemple, un breakpoint contenant `jitter.pc = 0x1234` fera reprendre l'exécution de l'émulateur à l'adresse 0x1234.